# Code to Scrape the Acting Landscape of the Most Popular Movies

In [3]:
import csv
import re
import requests
import threading
import json
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from pyvis.network import Network
import community.community_louvain as community_louvain

## Define Static Variables

In [4]:
# create url sections
preurl = "https://www.imdb.com/title/tt"
posturl = "/fullcredits?ref_=tt_cl_sm"

# define empty pandas dataframe
network_table = pd.DataFrame(columns =['Actor1', 'Actor2'])
network_table

,Actor1,Actor2


## Test Custom Functions

In [5]:
# import pre-defined functions from script
from scrapers import getURLs, retrieveCast, filterList, combinedRetrieve

# define test url and table
test_url = "https://www.imdb.com/title/tt0076759/fullcredits?ref_=tt_cl_sm" 
test_table = network_table

#test retrievePage function
test_list = retrieveCast(test_url)
test_list

Scraping cast list from: Star Wars (1977) 


[nan,
 'Mark Hamill',
 'Harrison Ford',
 'Carrie Fisher',
 'Peter Cushing',
 'Alec Guinness',
 'Anthony Daniels',
 'Kenny Baker',
 'Peter Mayhew',
 'David Prowse',
 'Phil Brown',
 'Shelagh Fraser',
 'Jack Purvis',
 'Alex McCrindle',
 'Eddie Byrne',
 'Drewe Henley',
 'Denis Lawson',
 'Garrick Hagon',
 'Jack Klaff',
 'William Hootkins',
 'Angus MacInnes',
 'Jeremy Sinden',
 'Graham Ashley',
 'Don Henderson',
 'Richard LeParmentier',
 'Leslie Schofield',
 'Rest of cast listed alphabetically:',
 'David Ankrum',
 'Mark Anthony Austin',
 'Scott Beach',
 'Lightning Bear',
 'Jon Berg',
 'Doug Beswick',
 'Paul Blake',
 'Janice Burchette',
 'Ted Burnett',
 'John Chapman',
 'Gilda Cohen',
 'Tim Condren',
 'Barry Copping',
 'Alfie Curtis',
 'Robert Davies',
 'Maria De Aragon',
 'Barbie Denham',
 'Frazer Diamond',
 'Peter Diamond',
 'Warwick Diamond',
 'Sadie Eden',
 'Kim Falkinburg',
 'Harry Fielder',
 'Anthony Forrest',
 'Ted Gagliano',
 'Salo Gardner',
 'Steve Gawley',
 'Barry Gnome',
 'Rusty Go

In [6]:
# test filterList function
filterList(test_list)

,Actor1,Actor2
0,Mark Hamill,Harrison Ford
0,Mark Hamill,Carrie Fisher
0,Mark Hamill,Peter Cushing
0,Mark Hamill,Alec Guinness
0,Mark Hamill,Anthony Daniels
...,...,...
0,Fred Wood,Larry Ward
0,Fred Wood,Diana Sadley Way
0,Fred Wood,Harold Weed
0,Fred Wood,Bill Weston


## Retrieve Sample URLs

In [7]:
#sample_urls = getURLs()

Save URLs to a JSON format

In [8]:
#out_file = open("sample_urls.json", "w")
#json.dump(sample_urls, out_file)

# we'll read from the JSON when re-running this project
with open("sample_urls.json") as file:
  sample_urls = list(json.load(file))
sample_urls

['https://www.imdb.com/title/tt0111161/fullcredits?ref_=tt_cl_sm',
 'https://www.imdb.com/title/tt0068646/fullcredits?ref_=tt_cl_sm',
 'https://www.imdb.com/title/tt0252487/fullcredits?ref_=tt_cl_sm',
 'https://www.imdb.com/title/tt0108052/fullcredits?ref_=tt_cl_sm',
 'https://www.imdb.com/title/tt0468569/fullcredits?ref_=tt_cl_sm',
 'https://www.imdb.com/title/tt0050083/fullcredits?ref_=tt_cl_sm',
 'https://www.imdb.com/title/tt0071562/fullcredits?ref_=tt_cl_sm',
 'https://www.imdb.com/title/tt0167260/fullcredits?ref_=tt_cl_sm',
 'https://www.imdb.com/title/tt5354160/fullcredits?ref_=tt_cl_sm',
 'https://www.imdb.com/title/tt0110912/fullcredits?ref_=tt_cl_sm',
 'https://www.imdb.com/title/tt7466810/fullcredits?ref_=tt_cl_sm',
 'https://www.imdb.com/title/tt1375666/fullcredits?ref_=tt_cl_sm',
 'https://www.imdb.com/title/tt0137523/fullcredits?ref_=tt_cl_sm',
 'https://www.imdb.com/title/tt0120737/fullcredits?ref_=tt_cl_sm',
 'https://www.imdb.com/title/tt0109830/fullcredits?ref_=tt_cl_

## Run Scraping Loop

In [10]:
dataframes = []

while len(sample_urls) > 0:
    
    threads = []
    
    if len(sample_urls) < 20:
        batch_urls = sample_urls
        sample_urls = []
    else:
        batch_urls = sample_urls[:20]
        del sample_urls[:20]
        
    for url in batch_urls:
        t = threading.Thread(target = combinedRetrieve, args = [url, dataframes])
        threads.append(t)
    
    for i in range(0, len(batch_urls)):
        threads[i].start()
        
    for i in range(0, len(batch_urls)):
        threads[i].join()

Scraping cast list from: Fight Club (1999) 
Scraping cast list from: 777 Charlie (2022) 
Scraping cast list from: Seven (1995) 
Scraping cast list from: The Good, the Bad and the Ugly (1966) 
Scraping cast list from: Episode #1.13070 (2007) 
Scraping cast list from: ¡Pedro Infante vive! (2007 TV Movie) 
Scraping cast list from: GoodFellas (1990) 
Scraping cast list from: Forrest Gump (1994) 
Scraping cast list from: Big Mon on Campus (1993) 
Scraping cast list from: One Flew Over the Cuckoo's Nest (1975) 
Scraping cast list from: Rocketry: The Nambi Effect (2022) Scraping cast list from: Star Wars: Episode V - The Empire Strikes Back (1980) 

Scraping cast list from: The Silence of the Lambs (1991) 
Scraping cast list from: The Green Mile (1999) 
Scraping cast list from: The Matrix (1999) 
Scraping cast list from: Star Wars (1977) 
Scraping cast list from: The Lord of the Rings: The Fellowship of the Ring (2001) 
Scraping cast list from: Interstellar (2014) 
Scraping cast list from: In

Scraping cast list from: For a Few Dollars More (1965) 
Scraping cast list from: Gone with the Wind (1939) 
Scraping cast list from: Laying a Marine Cable (1913) 
None
Error: Can't retrieve cast list (local variable 'cast_list' referenced before assignment)
Error: Can't retrieve actor table ('NoneType' object is not iterable)
Scraping cast list from: Drishyam (2015) 
Scraping cast list from: The Elephant Man (1980) 
Scraping cast list from: The Bridge on the River Kwai (1957) 
Scraping cast list from: Lock, Stock and Two Smoking Barrels (1998) 
Scraping cast list from: Monty Python and the Holy Grail (1975) 
Scraping cast list from: The Great Escape (1963) 
Scraping cast list from: Klaus (2019) 
Scraping cast list from: Ran (1985) 
Scraping cast list from: The Secret in Their Eyes (2009) 
Scraping cast list from: Unforgiven (1992) 
Scraping cast list from: Finding Nemo (2003) 
Scraping cast list from: Downfall (2004) 
Scraping cast list from: Chinatown (1974) 
Scraping cast list from: 

Scraping cast list from: Sarfarosh (1999) 
Scraping cast list from: Pink (III) (2016) 
Scraping cast list from: Her Sey Çok Güzel Olacak (1998) 
Scraping cast list from: Hera Pheri (2000) 
Scraping cast list from: La Grande Illusion (1937) 
Scraping cast list from: Kahaani (2012) 
Scraping cast list from: The Tashkent Files (2019) Scraping cast list from: The Circus (1928) 

Scraping cast list from: Udaan (2010) 
Scraping cast list from: Sholay (1975) Scraping cast list from: Thuppakki (2012) 
Scraping cast list from: Queen (2013) 

Scraping cast list from: Talvar (2015) 
Scraping cast list from: Masaan (2015) Scraping cast list from: Rififi (1955) 

Scraping cast list from: The Bandit (1996) 
Scraping cast list from: Black (2005) 
Scraping cast list from: The Princess Bride (1987) 
Scraping cast list from: Rain Man (1988) 
Scraping cast list from: Everything Everywhere All at Once (2022) 
Scraping cast list from: Kôga ranryû (2003) 
None
Error: Can't retrieve cast list (local variable

Scraping cast list from: Almost Famous (2000) Scraping cast list from: Harry Potter and the Prisoner of Azkaban (2004) 
Scraping cast list from: The Blues Brothers (1980) 

Scraping cast list from: Shrek (2001) 
Scraping cast list from: E.T. the Extra-Terrestrial (1982) 
Scraping cast list from: Arrival (II) (2016) 
Scraping cast list from: Edge of Tomorrow (2014) 
Scraping cast list from: Iron Man (2008) 
Scraping cast list from: Titanic (1997) 
Scraping cast list from: Star Trek (2009) 
Scraping cast list from: Thor: Ragnarok (2017) 
Scraping cast list from: In the Heat of the Night (1967) 
Scraping cast list from: The Wrestler (2008) 
Scraping cast list from: Brazil (1985) 
Scraping cast list from: This Is Spinal Tap (1984) 
Scraping cast list from: The Bourne Identity (2002) 
Scraping cast list from: Shaun of the Dead (2004) 
Scraping cast list from: Edward Scissorhands (1990) Scraping cast list from: Mystic River (2003) 

Scraping cast list from: Do the Right Thing (1989) 
Scrapin

Scraping cast list from: Back to the Future Part II (1989) 
Scraping cast list from: A Bronx Tale (1993) 
Scraping cast list from: Atonement (2007) 
Scraping cast list from: Cast Away (2000) 
Scraping cast list from: The Last Samurai (2003) 
Scraping cast list from: American Gangster (2007) 
Scraping cast list from: Skyfall (2012) 
Scraping cast list from: The Hobbit: The Desolation of Smaug (2013) 
Scraping cast list from: The Curious Case of Benjamin Button (2008) 
Scraping cast list from: Captain America: Civil War (2016) 
Scraping cast list from: Cabaret (1972) 
Scraping cast list from: Boyz n the Hood (1991) 
Scraping cast list from: Moon (2009) 
Scraping cast list from: Midnight Cowboy (1969) 
Scraping cast list from: I Saw the Devil (2010) 
Scraping cast list from: Willy Wonka & the Chocolate Factory (1971) 
Scraping cast list from: Gattaca (1997) 
Scraping cast list from: Misery (1990) 
Scraping cast list from: Mary Poppins (1964) 
Scraping cast list from: Being John Malkovich 

...

Scraping cast list from: Mama (I) (2013) 
Scraping cast list from: Trainwreck (2015) 
Scraping cast list from: Walking Tall (2004) 
Scraping cast list from: Zathura: A Space Adventure (2005) 
Scraping cast list from: RoboCop (2014) 
Scraping cast list from: Alice Through the Looking Glass (2016) 
Scraping cast list from: 47 Ronin (2013) 
Scraping cast list from: A Night at the Roxbury (1998) 
Scraping cast list from: Sleeping with the Enemy (1991) 
Scraping cast list from: Spenser Confidential (2020) 
Scraping cast list from: The Thing (I) (2011) 
Scraping cast list from: Hitman (I) (2007) 
Scraping cast list from: Adore (2013) 
Scraping cast list from: Color Out of Space (2019) 
Scraping cast list from: Endless Love (2014) 
Scraping cast list from: Why Him? (2016) 
Scraping cast list from: Eraser (1996) 
Scraping cast list from: Rogue (2007) Scraping cast list from: Jeepers Creepers (2001) 

Scraping cast list from: War (2007) 
Scraping cast list from: Reign of Fire (2002) 
Scraping c

Scraping cast list from: Sausage Party (2016) 
Scraping cast list from: One Night at McCool's (2001) 
Scraping cast list from: Machines in Motion: Living Machines (1984) 
Scraping cast list from: Ong Bak 2 (2008) 
Scraping cast list from: Chinku to yuurei sen (1968) 
Scraping cast list from: Section 60: Arlington National Cemetery (2008 TV Movie) 
None
Error: Can't retrieve cast list (local variable 'cast_list' referenced before assignment)
Error: Can't retrieve actor table ('NoneType' object is not iterable)
Scraping cast list from: Magic Mike (2012) 
Scraping cast list from: Open Grave (2013) 
Error: Can't retrieve actor table (No objects to concatenate)
Scraping cast list from: Lou (I) (2022) 
Scraping cast list from: Love (II) (2015) 
Scraping cast list from: Episode #1.9939 (2008) 
Scraping cast list from: The Nines (2007) 
Scraping cast list from: Dabangg (2010) 
Error: Can't retrieve actor table (No objects to concatenate)
Scraping cast list from: Mr. Nice Guy (1997) 
Scraping c

Scraping cast list from: The Lovebirds (2020) Scraping cast list from: Dear White People (2014) 

Scraping cast list from: Multiplicity (1996) 
Scraping cast list from: Extract (2009) 
Scraping cast list from: Solomon Kane (2009) 
Scraping cast list from: We Are Your Friends (2015) 
Scraping cast list from: Undisputed (2002) 
Scraping cast list from: 15 Minutes (2001) 
Scraping cast list from: The Kid (2000) 
Scraping cast list from: Ironclad (2011) 
Scraping cast list from: Gringo (2018) 
Scraping cast list from: Duplicity (2009) 
Scraping cast list from: Last Knights (2015) 
Scraping cast list from: Sky Captain and the World of Tomorrow (2004) 
Scraping cast list from: Welcome to the Punch (2013) 
Scraping cast list from: The Invisible (2007) 
Scraping cast list from: The Legend of Tomiris (2019) 
Scraping cast list from: Justice (2011) 
Scraping cast list from: Conquest of the Planet of the Apes (1972) 
Scraping cast list from: Greenberg (2010) 
Scraping cast list from: The Replacem

Scraping cast list from: To All the Boys: P.S. I Still Love You (2020) 
Scraping cast list from: Dragnet (1987) 
Scraping cast list from: Dream House (2011) 
Scraping cast list from: Nim's Island (2008) 
Scraping cast list from: V/H/S/2 (2013) 
Scraping cast list from: Father of the Bride Part II (1995) 
Scraping cast list from: Parental Guidance (2012) 
Scraping cast list from: Man of Tai Chi (2013) 
Scraping cast list from: World Trade Center (2006) 
Scraping cast list from: The Machine (I) (2013) 
Scraping cast list from: Thick as Thieves (I) (2009) 
Scraping cast list from: Intime ossessioni (2002 Video) 
Scraping cast list from: Grown Ups (I) (2010) 
Scraping cast list from: Five More Minutes (2009) 
Scraping cast list from: Omyl krále Salamouna (1999) 
Scraping cast list from: Episode #1.11 (2001) 
Scraping cast list from: K-9 (1989) 
Scraping cast list from: Unbreakable Souls (2015) 
Scraping cast list from: Erased (2012) 
Scraping cast list from: Memoirs of an Invisible Man (19

Scraping cast list from: California Man (1992) 
Scraping cast list from: The School for Good and Evil (2022) 
Scraping cast list from: Barbarella (1968) 
Scraping cast list from: Noah (2014) 
Scraping cast list from: Downsizing (2017) 
Scraping cast list from: Percy Jackson & the Lightning Thief (2010) 
Scraping cast list from: Teenage Mutant Ninja Turtles (2014) 
Scraping cast list from: 2012 (I) (2009) 
Scraping cast list from: Battleship (2012) 
Scraping cast list from: Vivarium (2019) 
Scraping cast list from: High Life (2018) 
Scraping cast list from: Pride and Prejudice and Zombies (2016) 
Scraping cast list from: Without Remorse (II) (2021) 
Scraping cast list from: The Addams Family (2019) 
Scraping cast list from: Oldboy (2013) Scraping cast list from: Hollow Man (2000) 

Scraping cast list from: The League of Extraordinary Gentlemen (2003) 
Scraping cast list from: The Princess Diaries 2 (2004) 
Scraping cast list from: Clash of the Titans (2010) 


## Aggregate into Final Network Table

In [12]:
network_table = pd.concat(dataframes)
network_table["value"] = 1

final_network_table = network_table.groupby(["Actor1","Actor2"], sort = False, as_index = False).sum()
final_network_table.sort_values('value', ascending = False)

,Actor1,Actor2,value
1225636,Cynthia LeBlanc,Elton LeBlanc,36
1225739,Elton LeBlanc,Cynthia LeBlanc,36
224622,Mickie McGowan,Sherry Lynn,29
224699,Sherry Lynn,Mickie McGowan,29
224776,Jack Angel,Mickie McGowan,25
...,...,...,...
10499223,Julia Ann West,Erik Smith,1
10499222,Julia Ann West,Paula Jones,1
10499221,Julia Ann West,Marisa Miller,1
10499220,Julia Ann West,Xavier Hernandez,1


## Filter Data For Performance

In [25]:
# create table of total connections
filter_table = network_table.groupby(["Actor1"], sort = False, as_index = False).sum('value')
actors = list(filter_table.query('value > 100')["Actor1"])

# filter table to only include these actors
final_network_table = final_network_table.query('Actor1 in @actors and Actor2 in @actors')
final_network_table.sort_values('value', ascending = False)

,Actor1,Actor2,value
1225739,Elton LeBlanc,Cynthia LeBlanc,36
1225636,Cynthia LeBlanc,Elton LeBlanc,36
224699,Sherry Lynn,Mickie McGowan,29
224622,Mickie McGowan,Sherry Lynn,29
224623,Mickie McGowan,Jack Angel,25
...,...,...,...
10609247,Jake Picking,Pamela Figueiredo,1
10609246,Jake Picking,Mugisha Feruzi,1
10609245,Jake Picking,Tayla Fernandez,1
10609244,Jake Picking,Mark Falvo,1


## Create Network Visualisation

In [26]:
G = nx.from_pandas_edgelist(final_network_table, 
                            source = "Actor1", 
                            target = "Actor2", 
                            edge_attr = "value", 
                            create_using = nx.Graph())

In [ ]:
plt.figure(figsize=(10,10))
pos = nx.kamada_kawai_layout(G)
nx.draw(G, with_labels = True, node_color = 'skyblue', edge_cmap = plt.cm.Blues, pos = pos)
plt.show()

## Create Second Network Visualisation

In [ ]:
net = Network(notebook = True, width="1000px", height="700px", bgcolor='#222222', font_color='white')

node_degree = dict(G.degree)

#Setting up node size attribute
nx.set_node_attributes(G, node_degree, 'size')

net.from_nx(G)
net.show("tt.html")